In [5]:
!pip install pandas yfinance sgs python-bcb

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import pandas as pd
import yfinance as yf
from bcb import sgs
import time

In [7]:
#Comecar com dolar, IBOVESP e S&P500(bolsa americana)
indicadores = ['^BVSP', 'BRL=X', '^GSPC']
dados_yahoo = yf.download(indicadores, start="2016-01-01")['Close']
dados_yahoo.columns = ['Dolar', 'Ibovespa', 'SP500']
#agora pegar indicadores do banco central(SELIC, IPCA e IPGM)
codigos_bc = {
    'Selic': 432, 
    'IPCA': 433, 
    'IGPM': 189
}
try:
    # Primeiro de 2016 a 2020(deu erro pegar tudo direto)
    bc_parte1 = sgs.get(codigos_bc,start="2016-01-01",end="2020-12-31")
    time.sleep(2) 
    #AGora os ultimos anos
    bc_parte2 = sgs.get(codigos_bc, start="2021-01-01")
    #criamos a parte do banco central completa usando a funcao concat do pandas(colocando uma embaixo da outra)
    dados_bc_completo = pd.concat([bc_parte1, bc_parte2], axis=0)
except Exception as e:
    print(f"Deu erro!{e}")
    dados_bc_completo = pd.DataFrame()
if not dados_bc_completo.empty:
    # Agora juntamos ambas as tabelas colocando uma do lado da outra
    df_final = pd.concat([dados_yahoo, dados_bc_completo], axis=1)
    #A inflacao eh dada so 1 vez a cada 45 dias, precisamos preencher os outros dias
    df_final = df_final.ffill()
    #Remove as linhas problematicas
    df_final = df_final.dropna()
else:
    print("Deu tudo errado")
print("Base de dados de 2016 a 2025")
print(df_final.tail())


/tmp/ipykernel_89742/3209979044.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dados_yahoo = yf.download(indicadores, start="2016-01-01")['Close']
[*********************100%***********************]  3 of 3 completed


Base de dados de 2016 a 2025
             Dolar     Ibovespa        SP500  Selic  IPCA  IGPM
Date                                                           
2025-11-20  5.3286  155381.0000  6538.759766   15.0  0.09 -0.36
2025-11-21  5.3323  154770.0000  6602.990234   15.0  0.09 -0.36
2025-11-22  5.3323  154770.0000  6602.990234   15.0  0.09 -0.36
2025-11-23  5.3323  154770.0000  6602.990234   15.0  0.09 -0.36
2025-11-24  5.3889  155240.9375  6708.439941   15.0  0.09 -0.36
